In [ ]:
### Best schedule for a student

### Time Preferences
### Teacher Preferences
### Building Preferences
### Load Preferences

### 

In [93]:
import numpy as np
from collections import defaultdict
import random
import math

In [8]:
DAYS = {"M": "000", "T": "001", "W": "010", "TH": "011", "F":"100"}

In [9]:
DAYS_INV = dict((v, k) for k, v in DAYS.items())

In [10]:
TIMES = {8: "0000", 9: "0001", 10: "0010", 11: "0011", 12: "0100", 13: "0101", 14: "0110", 15: "0111", 16: "1000"}

In [218]:
TERMS = {"Fall" : 0, "Winter": 1}

In [11]:
TIMES_INV = dict((v, k) for k, v in TIMES.items())

In [12]:
LENGTHS = {1: "00", 1.5: "01", 2: "10"}

In [13]:
LENGTHS_INV = dict((v, k) for k, v in LENGTHS.items())

In [14]:
ROOMS = {"R1": "0000", "R2": "0001", "R3": "0010", "R4": "0011", "R5": "0100", "R6": "0101", "R7": "0110", "R8": "0111", "R9": "1000"}

In [15]:
TEACHER = {"T1": "0000", "T2": "0001", "T3": "0010", "T4": "0011", "T5": "0100", "T6": "0101", "T7": "0110", "T8": "0111", "T9": "1000"}

In [16]:
CLASS = {"C1": "0000", "C2": "0001", "C3": "0010", "C4": "0011", "C5": "0100", "C6": "0101", "C7": "0110", "C8": "0111", "C9": "1000"}

In [17]:
DICTS = {"DAYS": DAYS, "TIMES": TIMES, "LENGTHS": LENGTHS, "ROOMS": ROOMS, "TEACHER": TEACHER, "CLASS": CLASS,}

In [18]:
GENE_LENGTHS = {"DAYS": len(DAYS["M"]), "TIMES": len(TIMES[8]), "LENGTHS": len(LENGTHS[1]), "ROOMS": len(ROOMS["R1"]), "TEACHER": len(TEACHER["T1"]),
                "CLASS": len(CLASS["C1"]),}

In [19]:
def get_gene_bounds(GENE_LENGTHS):
    bounds = {}
    start = 0
    for key in GENE_LENGTHS.keys():
        end = GENE_LENGTHS[key] + start
        bounds[key] = (start, end)
        start += GENE_LENGTHS[key]
    return bounds

BOUNDS = get_gene_bounds(GENE_LENGTHS)        

In [20]:
class BinFuncs:
    @staticmethod
    def get_slot(s1):
        time_len = GENE_LENGTHS["TIMES"]
        day_len = GENE_LENGTHS["DAYS"]
        length_len = GENE_LENGTHS["LENGTHS"]
        
        d_s, d_e = BOUNDS["DAYS"]
        t_s, t_e = BOUNDS["TIMES"]
        l_s, l_e = BOUNDS["LENGTHS"]
        
        day_1 = DAYS_INV[s1[d_s:d_e]]
        time_1 = TIMES_INV[s1[t_s:t_e]]
        length_1 = LENGTHS_INV[s1[l_s:l_e]]
        
        return (day_1, time_1, length_1)
    
    @staticmethod
    def is_overlapping(tuple1, tuple2):
        day_1, time_1, length_1 = tuple1
        day_2, time_2, length_2 = tuple2
        return Event(day_1, time_1, length_1).is_overlapping(Event(day_2, time_2, length_2))        

In [189]:
import random
class GA:
    # Swap the genes of two parents
    @staticmethod
    def swap(p1, p2, attr):              
        p1 = list(p1)
        p2 = list(p2)

        start_index, end_index = BOUNDS[attr]

        p1[start_index:end_index], p2[start_index:end_index] = p2[start_index:end_index], p1[start_index:end_index]

        return ''.join(p1) , ''.join(p2)
    
    # Get a random mutation on time attr
#     @staticmethod
#     def mutate_time(p1):
#         p1 = list(p1)
        
#         keys_list = list(BOUNDS.keys())[:2]
        
#         attr = random.choice(keys_list)
#         new_val = random.choice(list(DICTS[attr].values()))
#         start, end = BOUNDS[attr]
        
#         p1[start:end] = new_val
        
#         return ''.join(p1)
    
    def mutate_time(event):
        day_list = list(DAYS.keys())
        time_list = list(TIMES.keys())
        
        event.time = random.choice(time_list)
        return event
    
    @staticmethod
    def mutate_schedule(schedule, prob=0.5):
        for i, event in enumerate(schedule.schedule):
            if np.random.rand() < prob:
                mutated_event = GA.mutate_time(event)
                schedule.schedule[i] = mutated_event
        
            
    @staticmethod
    def inverse_fitness(s_bin, ITEM_LEN):
        total_conflicts = 0
        days = defaultdict(list)
        index = 0

        for i in range(ITEM_LEN, len(s_bin), ITEM_LEN):
            event_one = BinFuncs.get_slot(s_bin[i:i+ITEM_LEN])
            for j in range(i-ITEM_LEN, -1, -ITEM_LEN):
                event_two = BinFuncs.get_slot(s_bin[j:j+ITEM_LEN])
                conflict = BinFuncs.is_overlapping(event_one, event_two)
                if conflict:
                    total_conflicts += 1
                    


        return 1./(1. + total_conflicts)
    
    @staticmethod
    def time_prefs(s_bin, time_prefs, ITEM_LEN):
        PREF = 0.01
        penalty = 0
        
        for i in range(ITEM_LEN, len(s_bin), ITEM_LEN):
            event_one = BinFuncs.get_slot(s_bin[i:i+ITEM_LEN])
            for pref_event in (time_prefs):
                conflict = BinFuncs.is_overlapping(event_one, BinFuncs.get_slot(pref_event.get_genome()))
                if conflict:
                    penalty -= PREF
                    break
                    
        return penalty
    
    # Blending two parents and producing two children.
    # https://www.researchgate.net/publication/282269189_An_empirical_comparison_of_two_crossover_operators_in_real-coded_genetic_algorithms_for_constrained_numerical_optimization_problems
    @staticmethod
    def blend_crossover(s1, s2):
        child_one = Schedule(event_list=s1.schedule)
        child_two = Schedule(event_list=s2.schedule)
        for i, (x1, x2) in enumerate(zip(s1.schedule, s2.schedule)):
            min_start = min(x1.start, x2.start)
            max_start = max(x1.start, x2.start)

            diff = max_start-min_start
            alpha = 0.5

            i_start = math.floor(min_start - (diff * alpha))
            i_end = math.ceil(max_start + (diff * alpha))

            # This part should prob assign intervals bounds to key with least difference
            # instead of current implementation
            if(i_start not in TIMES.keys()):
                i_start = min(list(TIMES.keys()))

            if(i_end not in TIMES.keys()):
                i_end = max(list(TIMES.keys()))

            parent_time_range = range(i_start, i_end+1)

            time_one = random.choice(parent_time_range)
            time_two = random.choice(parent_time_range)

            parent_day_range = [x1.day, x2.day]

            day_one = random.choice(parent_day_range)
            day_two = random.choice(parent_day_range)

            child_one.schedule[i] = Event(day_one, time_one, x1.len)
            child_two.schedule[i] = Event(day_two, time_two, x2.len)
        return child_one, child_two
        
        
    # Get index list of sorted fitness
    @staticmethod
    def fitness_indexes(schedules, time_prefs):
        fitness_values = []
        for s in schedules:
            fitness_values.append(s.get_fitness(time_prefs)
        return np.flip(np.argsort(fitness_values))
    
    @staticmethod
    def population_fitness(pop, time_prefs):  
        total_fitness = 0
                            
        for s in pop:
            total_fitness +=  
            total_fitness += s
        return total_fitness   

In [208]:
def roulette_selection(pop, prefs):
    total = sum([s.get_fitness(prefs) for s in pop])
    
    selection_probs = [s.get_fitness(prefs)/total for s in pop]
    
    new_pop = np.random.choice(pop, size=len(pop)-2, replace=False, p=selection_probs)
    
    return new_pop.tolist()

In [217]:
pop = []
for x in range(500):
    pop.append(Schedule(10))
    
algorithm(pop, empty_morns, 50)

Current fitness 150.9416666666665
Current fitness 151.06666666666635
Current fitness 150.50761904761893
Current fitness 149.01095238095206
Current fitness 147.0776190476188
Current fitness 146.02809523809512
Current fitness 144.83190476190455
Current fitness 144.15571428571405
Current fitness 143.03904761904747
Current fitness 143.15619047619037
Current fitness 142.7433333333332
Current fitness 142.81380952380925
Current fitness 142.64380952380927
Current fitness 142.24880952380943
Current fitness 142.45880952380924
Current fitness 142.0059523809523
Current fitness 141.6192857142856
Current fitness 142.008095238095
Current fitness 141.54142857142838
Current fitness 141.63857142857123
Current fitness 142.32571428571396
Current fitness 143.0407142857141
Current fitness 143.58071428571415
Current fitness 144.2140476190477
Current fitness 144.84404761904756
Current fitness 145.54404761904752
Current fitness 146.20738095238082
Current fitness 146.80071428571412
Current fitness 147.497380952

In [212]:
def algorithm(initial_pop, time_prefs, generations=10):
    current_pop = initial_pop
    best_pop = current_pop
    best_score = 0
    best_gen = 0
    for i in range(generations):
        # Selection
        fit_order = GA.fitness_indexes(current_pop, time_prefs)
        p1 = current_pop[fit_order[0]]
        p2 = current_pop[fit_order[1]]

        # Crossover
        c1, c2 = GA.blend_crossover(p1, p2)

        # Mutation
        GA.mutate_schedule(c1)
        GA.mutate_schedule(c2)
        
        current_pop = roulette_selection(current_pop, time_prefs)
        
        current_pop.append(c1)
        current_pop.append(c2)
        
        
        
        fitness_score = GA.population_fitness(current_pop, time_prefs)
        if(fitness_score > best_score):
            best_pop = current_pop
            best_score = fitness_score
            best_gen = i
        
        print(f"Current fitness {fitness_score}")
        
    pop_order = GA.fitness_indexes(best_pop, time_prefs)
    best_schedule = best_pop[pop_order[0]]
    best_schedule.schedule.sort(key=lambda x: (x.day, x.start))
    print(best_schedule)
    print(best_schedule.get_fitness(time_prefs))

In [172]:
x1 = Schedule(10)
x2 = Schedule(10)
c1, c2 = GA.blend_crossover(x1, x2)
print(c1)
print(c2)

range(8, 15)
range(8, 16)
range(9, 13)
range(8, 16)
range(8, 17)
range(11, 12)
range(8, 17)
range(8, 17)
range(8, 17)
range(8, 17)
Starts at 14 for 2.0 hours on W 
Starts at 10 for 2.0 hours on W 
Starts at 12 for 2.0 hours on F 
Starts at 8 for 2.0 hours on M 
Starts at 11 for 2.0 hours on TH 
Starts at 11 for 2.0 hours on TH 
Starts at 14 for 2.0 hours on W 
Starts at 11 for 2.0 hours on W 
Starts at 15 for 2.0 hours on F 
Starts at 14 for 2.0 hours on W 

Starts at 13 for 2.0 hours on W 
Starts at 12 for 2.0 hours on W 
Starts at 9 for 2.0 hours on F 
Starts at 14 for 2.0 hours on M 
Starts at 8 for 2.0 hours on M 
Starts at 11 for 2.0 hours on T 
Starts at 11 for 2.0 hours on F 
Starts at 14 for 2.0 hours on W 
Starts at 9 for 2.0 hours on F 
Starts at 10 for 2.0 hours on W 



In [121]:
d1 = Event("M", 9, 2)
d2 = Event("W", 10, 2)



0.45
0.2833333333333333
0.2333333333333333
0.2
0.1833333333333333
0.15000000000000002
0.15000000000000002
0.1333333333333333
0.1
0.09999999999999998


In [23]:
    #     print(days)

    #     for key in days.keys():
    #         events = days[key]

    #         for i in range(1, len(events)):
    #             event_one = BinFuncs.get_slot(key+events[i])
    #             print(f"Outer {event_one} index {i}")

    #             for j in range(i-1, -1, -1):
    #                 event_two = BinFuncs.get_slot(key+events[j])
    #                 print(f"Inner {event_two} index {j}")
    #                 conflict = BinFuncs.is_overlapping(event_one, event_two)
    #                 if conflict:
    #                     print("conflict")
    #                     total_conflicts += 1
    #     for key in days.keys():
    #         days[key] = sorted(days[key])

    #         ongoing = 0
    #         for event in days[key]:


    #     for event s.schedule:
    #         print(eventOne)
    #         for eventTwo in s.schedule:
    #             if eventOne != eventTwo:
    #                 if eventOne.is_overlapping(eventTwo):
    #                     total_conflicts+= 1

In [24]:
p1 = Event("M", 11, 1)
print(p1)
d, t, l = BinFuncs.get_slot(GA.mutate_time(p1.get_genome()))
print(Event(d, t, l))
d, t, l = BinFuncs.get_slot(GA.mutate_time(p1.get_genome()))
print(Event(d, t, l))
d, t, l = BinFuncs.get_slot(GA.mutate_time(p1.get_genome()))
print(Event(d, t, l))

NameError: name 'Event' is not defined

In [25]:
p1 = Event("M", 11, 1)
p1.teacher = "T2"
p2 = Event("T", 10, 1)

p3 = Event("W", 12, 2)
print(p1.get_genome(), p2.get_genome())
p1_cross, p2_cross = GA.swap(p1.get_genome(), p2.get_genome(), "DAYS")
print(p1_cross, p2_cross)
# print(LENGTHS[1.5])
# print(LENGTHS[1])

NameError: name 'Event' is not defined

In [26]:
class Event:
    def __init__(self, day, start, length):
        self.start = int(start)
        self.len = float(length)
        self.day = day
        
        self.teacher = "T1"
        self.class_name = "C1"
        self.room = "R1"
        
    def is_overlapping(self, event):
        end = self.start + self.len
        other_end = event.start + event.len
        
        end_during = end > event.start and end <= other_end
        start_during = self.start >= event.start and self.start < other_end
        same_day = (self.day == event.day)
        
        return (end_during or start_during) and same_day

    
    def __str__(self):
        return f"Starts at {self.start} for {self.len} hours on {self.day}"
    
    def time_gene(self):
        return DAYS[self.day] + TIMES[self.start] + LENGTHS[self.len]
    
    def get_genome(self):
        genome = ""
        genome += self.time_gene()
        genome += ROOMS[self.room]
        genome += TEACHER[self.teacher]
        genome += CLASS[self.class_name]
        return genome

In [191]:
class Schedule:
    def __init__(self, events = 10, event_list =None):
        
        if event_list is not None:
            self.schedule = event_list
        else:
            self.schedule = []

            for x in range(events):
                # Get random 
                d, t, l = random_start_time()
                self.schedule.append(Event(d, t, l))            
            
    def __str__(self):
        rep = ""
        for event in self.schedule:
            rep = rep + str(event) + " "
            rep = rep + "\n"
        return rep
    
    def print_genome(self):
        genome = ""
        for event in self.schedule:
            print(event.get_genome())
            
    def get_as_bin(self):
        genome = ""
        for event in self.schedule:
            genome += event.get_genome()
        return genome
    
    def get_fitness(self, time_prefs):
        s = 0
        s += GA.inverse_fitness(self.get_as_bin(), len(self.schedule[0].get_genome()))
        s += GA.time_prefs(self.get_as_bin(), time_prefs, len(self.schedule[0].get_genome())) 
        return s

In [31]:
len(s.schedule[0].get_genome())

NameError: name 's' is not defined

In [48]:
s = Schedule(events=10)
best_score = 0
best_s = []

x = 0
x_best = 0
while(best_score < 1.0 and x < 5000):
    s = Schedule(events=15)
    score = GA.inverse_fitness(s.get_as_bin(), len(s.schedule[0].get_genome()))
    score += GA.time_prefs(s.get_as_bin(), empty_morns, len(s.schedule[0].get_genome()))
    print(score)
    if(score > best_score):
        best_score = score
        best_s = s
        x_best = x
    x += 1
s.schedule.sort(key=lambda x: (x.day, x.start))
best_s.schedule.sort(key=lambda x: (x.day, x.start))
print(best_s)

print(f"Iterations: {x}, Score {best_score} Generation {x_best}")
                               
# s_2 = Schedule(events=10)
# s_3 = Schedule(events=10)
# s.schedule = []
# s.schedule.append(Event("M", 12, 2))
# s.schedule.append(Event("F", 12, 2))
# s.schedule.append(Event("M", 8, 1))
# s.schedule.append(Event("M",8, 2))
# s.schedule.append(Event("T",8, 2))

# s.schedule.append(Event("T",8, 2))
# s.schedule.append(Event("T",8, 2))
# print(GA.inverse_fitness(s.get_as_bin(), len(s.schedule[0].get_genome())))
# empty_morns = [Event(x, 8, 2) for x in DAYS.keys()]
# print(GA.time_prefs(s.get_as_bin(), empty_morns,len(s.schedule[0].get_genome())))
      
# print(GA.inverse_fitness(s_2.get_as_bin(), len(s.schedule[0].get_genome())))
# print(GA.inverse_fitness(s_3.get_as_bin(), len(s.schedule[0].get_genome())))

# print(s)
# print(s_2)
# print(s_3)


0.042857142857142844
-0.0888888888888889
0.06666666666666665
0.09999999999999998
-0.15714285714285714
-0.007142857142857173
0.1
-0.033333333333333354
0.04999999999999999
0.016666666666666635
0.016666666666666635
0.09999999999999998
0.075
-0.025000000000000022
-0.08333333333333334
0.0
-0.18888888888888888
0.1
0.1
0.11666666666666665
-0.05909090909090911
0.016666666666666635
0.1
-0.025000000000000022
-0.03888888888888892
0.15
0.04999999999999999
-0.04999999999999999
-0.016666666666666677
0.0
-0.05714285714285716
-0.007142857142857173
-0.05909090909090911
0.024999999999999994
-0.05909090909090911
-0.0888888888888889
0.09285714285714285
0.15000000000000002
0.1
-0.025000000000000022
0.04999999999999999
0.1
0.0611111111111111
0.024999999999999994
-0.16666666666666669
-0.033333333333333354
-0.22499999999999998
0.04999999999999999
0.1
-0.025000000000000022
-0.007142857142857173
-0.05714285714285716
0.2
0.0611111111111111
-0.05000000000000002
-0.009090909090909094
0.09285714285714285
-0.0250000

-0.10714285714285715
-0.05714285714285716
0.1
0.0111111111111111
-0.13333333333333333
0.016666666666666635
-0.03888888888888892
-0.05909090909090911
0.042857142857142844
0.016666666666666635
-0.125
-0.025000000000000022
-0.08333333333333334
0.042857142857142844
0.15000000000000002
0.016666666666666635
0.09285714285714285
-0.1
-0.25909090909090904
-0.0888888888888889
0.09090909090909091
0.016666666666666635
-0.15714285714285714
0.1833333333333333
-0.09999999999999998
-0.1388888888888889
0.0
0.0
-0.15
0.0625
-0.05714285714285716
-0.125
0.0
0.09285714285714285
0.1
0.042857142857142844
-0.0666666666666667
0.04999999999999999
-0.007142857142857173
-0.05000000000000002
-0.21666666666666667
-0.009090909090909094
0.04999999999999999
-0.08333333333333334
0.042857142857142844
-0.016666666666666677
-0.05000000000000002
-0.033333333333333354
-0.025000000000000022
-0.18888888888888888
-0.025000000000000022
0.06666666666666665
0.0611111111111111
-0.07500000000000001
0.04999999999999999
-0.0750000000

-0.0666666666666667
0.06666666666666665
-0.03888888888888892
0.14285714285714285
-0.0888888888888889
-0.009090909090909094
0.05
-0.025000000000000022
0.1
0.033333333333333326
-0.03888888888888892
-0.033333333333333354
0.042857142857142844
-0.03888888888888892
-0.05714285714285716
-0.125
-0.125
-0.15
-0.05714285714285716
-0.10714285714285715
0.04999999999999999
0.016666666666666635
-0.12307692307692308
0.0111111111111111
0.04999999999999999
-0.08333333333333334
0.0111111111111111
-0.033333333333333354
0.09285714285714285
-0.125
0.04999999999999999
0.024999999999999994
0.016666666666666635
0.0111111111111111
-0.08333333333333334
-0.033333333333333354
-0.016666666666666677
-0.025000000000000022
0.016666666666666635
-0.05714285714285716
-0.07500000000000001
0.016666666666666635
0.016666666666666635
-0.0888888888888889
-0.125
0.0111111111111111
0.024999999999999994
-0.08333333333333334
-0.1
-0.05000000000000002
0.016666666666666635
-0.0666666666666667
0.04999999999999999
-0.175
-0.007142857

0.016666666666666663
0.016666666666666635
0.0111111111111111
-0.025000000000000022
-0.05714285714285716
0.06666666666666665
0.0
-0.007142857142857173
-0.05714285714285716
-0.359090909090909
-0.025000000000000022
0.016666666666666635
-0.08333333333333336
-0.03888888888888892
-0.10714285714285715
-0.033333333333333354
0.0
0.016666666666666635
-0.175
-0.03888888888888892
0.0111111111111111
0.16666666666666666
-0.18888888888888888
-0.20909090909090908
0.016666666666666635
0.09999999999999998
0.04999999999999999
-0.033333333333333354
-0.04999999999999999
0.09285714285714285
-0.04999999999999999
-0.016666666666666677
-0.05714285714285716
-0.033333333333333354
0.06666666666666665
-0.19999999999999998
-0.025000000000000022
0.033333333333333326
0.0
0.15000000000000002
0.2
0.026923076923076925
0.04999999999999999
0.016666666666666635
-0.025000000000000022
0.11666666666666665
-0.025000000000000022
0.09090909090909091
0.016666666666666635
-0.025000000000000022
-0.03888888888888892
-0.0500000000000

0.1333333333333333
-0.1
-0.05909090909090911
-0.025000000000000022
-0.05000000000000002
0.0111111111111111
-0.10714285714285715
0.1
-0.0666666666666667
-0.05909090909090911
0.04999999999999999
0.04090909090909091
0.0
-0.36666666666666664
-0.025000000000000022
-0.10714285714285715
-0.033333333333333354
-0.0888888888888889
-0.11666666666666668
-0.05714285714285716
-0.07500000000000001
-0.11666666666666668
0.05
0.024999999999999994
0.075
-0.05909090909090911
0.15000000000000002
-0.23888888888888887
0.024999999999999994
0.024999999999999994
-0.18888888888888888
0.09285714285714285
0.042857142857142844
-0.007142857142857173
0.11666666666666665
0.016666666666666635
-0.1388888888888889
-0.125
-0.05000000000000002
0.08333333333333331
0.06666666666666665
0.09285714285714285
-0.033333333333333354
0.024999999999999994
-0.08333333333333334
0.15000000000000002
-0.007142857142857173
0.06666666666666665
0.016666666666666635
-0.04999999999999999
-0.025000000000000022
0.04999999999999999
0.024999999999

0.016666666666666635
-0.03888888888888892
0.016666666666666635
0.06666666666666665
-0.07500000000000001
0.042857142857142844
-0.025000000000000022
0.042857142857142844
-0.025000000000000022
0.15
0.14285714285714285
-0.1388888888888889
0.04999999999999999
-0.175
-0.03888888888888892
0.042857142857142844
-0.03888888888888892
0.2333333333333333
-0.025000000000000022
-0.11153846153846156
0.0111111111111111
0.11666666666666665
0.09285714285714285
-0.007142857142857173
-0.08333333333333334
0.16666666666666666
-0.07500000000000001
-0.05714285714285716
-0.28888888888888886
-0.1
0.042857142857142844
-0.05714285714285716
-0.07500000000000001
-0.08750000000000002
0.09285714285714285
-0.0888888888888889
0.15
0.0
0.016666666666666635
0.1833333333333333
-0.1090909090909091
0.11666666666666665
0.0
-0.1388888888888889
0.08333333333333331
0.0
0.08333333333333333
-0.03888888888888892
0.04090909090909091
-0.07500000000000001
-0.0888888888888889
0.0111111111111111
0.1
0.09999999999999998
-0.06666666666666

0.15000000000000002
0.016666666666666635
0.0
-0.007142857142857173
-0.25909090909090904
0.09285714285714285
-0.125
-0.1
0.06666666666666665
-0.125
0.09285714285714285
-0.0785714285714286
-0.07500000000000001
-0.05000000000000002
0.04999999999999999
0.0
0.016666666666666635
-0.08333333333333334
-0.125
0.04999999999999999
-0.10714285714285715
-0.10714285714285715
-0.0730769230769231
0.15
0.06666666666666665
0.07692307692307693
0.2
0.2
0.0
0.1
0.0
0.09999999999999998
0.075
0.15
0.14285714285714285
-0.1
0.016666666666666635
0.06666666666666665
-0.07500000000000001
-0.007142857142857173
-0.04999999999999999
0.024999999999999994
0.0111111111111111
-0.007142857142857173
0.11666666666666665
0.042857142857142844
0.024999999999999994
0.024999999999999994
0.04999999999999999
0.1
0.0111111111111111
0.0
0.0
-0.21666666666666667
0.09999999999999998
-0.125
-0.1590909090909091
-0.033333333333333354
0.09999999999999998
0.026923076923076925
-0.1
-0.15
-0.007142857142857173
-0.05909090909090911
0.1111111

-0.03888888888888892
-0.05909090909090911
0.1333333333333333
0.05
-0.007142857142857173
0.042857142857142844
0.15000000000000002
-0.18888888888888888
-0.13333333333333333
0.1111111111111111
-0.0888888888888889
-0.03888888888888892
-0.0730769230769231
0.15
0.09285714285714285
0.06666666666666665
-0.007142857142857173
0.09285714285714285
0.06666666666666665
0.06666666666666665
-0.009090909090909094
-0.05909090909090911
0.1333333333333333
-0.025000000000000022
-0.20909090909090908
-0.2785714285714286
0.06666666666666665
-0.22499999999999998
-0.04999999999999999
0.04999999999999999
0.016666666666666635
-0.0666666666666667
0.042857142857142844
-0.0888888888888889
0.0625
-0.1090909090909091
-0.1388888888888889
-0.1388888888888889
-0.20909090909090908
0.0
-0.007142857142857173
-0.21666666666666667
-0.125
0.042857142857142844
-0.007142857142857173
-0.007142857142857173
-0.23888888888888887
0.04999999999999999
0.09285714285714285
0.04999999999999999
-0.08333333333333334
0.0111111111111111
-0.07

-0.009090909090909094
0.024999999999999994
-0.04999999999999999
0.1833333333333333
0.016666666666666635
0.0611111111111111
-0.03888888888888892
-0.10714285714285715
-0.025000000000000022
0.1
-0.04444444444444445
0.15
-0.033333333333333354
0.06666666666666665
-0.03888888888888892
0.0611111111111111
-0.04999999999999999
-0.10714285714285715
0.09285714285714285
-0.02857142857142858
0.0
0.06666666666666665
-0.08333333333333334
-0.007142857142857173
0.04999999999999999
-0.10714285714285715
0.07692307692307693
-0.0666666666666667
0.04999999999999999
-0.08333333333333334
-0.125
-0.15
0.06666666666666665
-0.07500000000000001
-0.009090909090909094
0.0
0.04090909090909091
-0.033333333333333354
0.2333333333333333
0.1833333333333333
0.016666666666666635
-0.09999999999999998
0.04090909090909091
0.0111111111111111
0.016666666666666635
-0.025000000000000022
0.2
0.042857142857142844
0.024999999999999994
0.024999999999999994
-0.07500000000000001
0.042857142857142844
0.2
0.0
0.09285714285714285
0.042857